# Pre-Analysis of Collected Twitter Data

In [1]:
from pyspark.sql.functions import col
from pyspark.sql.functions import split, explode,expr
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from pyspark.sql.functions import countDistinct
import pandas as pd
from pyspark import SparkContext
SparkContext.setSystemProperty('spark.executor.memory', '2g')
sc = SparkContext("local", "App Name")
spark = SparkSession.builder.getOrCreate()

In [2]:
#Load tweets as Dataframe to Spark
tweetsDF = spark.read.json("file:///Users/Laith/Downloads/tweetdata.json")

#Print the schema/data structure in a tree format
tweetsDF.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- date: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- influencer: long (nullable = true)
 |-- is_quote_status: boolean (nullable = true)
 |-- is_verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- original_tweet: long (nullable = true)
 |-- place: struct (nullable = true)
 |    |-- bounding_box: struct (nullable = true)
 |    |    |-- coordinates: array (nullable = true)
 |    |    |    |-- element: array (containsNull = t

# Understanding the Data

In [3]:

print("---------------------User Location Counting --------------------------------")
#Get top 20 places 
tweetsDF.select("place.country").groupBy("country").count().sort(col("count").desc()).show(20)

#Get top 21 user locations
tweetsDF.groupBy("location").count().sort(col("count").desc()).show(21)
#Get top 10 locations within the UK
tweetsDF.groupBy("location").count().sort(col("count").desc()).where("location LIKE '%UK%' OR location LIKE '%London%' OR location LIKE '%United Kingdom%' OR location LIKE '%England%' OR location LIKE '%Scotland%' OR location LIKE '%Wales%' OR location LIKE '%Ireland%'").show(10)

---------------------User Location Counting --------------------------------
+--------------+-----+
|       country|count|
+--------------+-----+
|          null|29480|
|United Kingdom|  293|
| United States|   64|
|        Italia|   21|
|   Royaume-Uni|   16|
|        España|   12|
|         India|   10|
|        Canada|    8|
|     Australia|    6|
|        Hellas|    6|
|       Ireland|    5|
|        Brasil|    5|
|   Deutschland|    5|
|       Germany|    5|
|        France|    5|
|      Malaysia|    4|
|       Nigeria|    4|
|         Spain|    3|
|        Mexico|    3|
|       Ecuador|    2|
+--------------+-----+
only showing top 20 rows

+--------------------+-----+
|            location|count|
+--------------------+-----+
|                null|10337|
|     London, England|  646|
|              London|  511|
|      United Kingdom|  416|
|England, United K...|  336|
|                  UK|  313|
|       United States|  225|
|              France|  161|
|             England|  14

In [4]:
 #Language Counting 
print("---------------------User's Languages Counting --------------------------------")
    #Get user languages
tweetsDF.groupBy("tweet_lang").count().sort(col("count").desc()).show(10)
    #Get English tweets
tweetsEn = tweetsDF.where("tweet_lang=='en'")

---------------------User's Languages Counting --------------------------------
+----------+-----+
|tweet_lang|count|
+----------+-----+
|        en|20811|
|        it| 1810|
|        fr| 1663|
|       und| 1434|
|        es| 1285|
|        de|  980|
|        nl|  796|
|        pt|  347|
|        ja|  320|
|        ca|   99|
+----------+-----+
only showing top 10 rows



In [5]:
#Check tweets that are not responses or quotes - original tweets
tweetsEn.groupBy("original_tweet").count().sort(col("count").desc()).show(10)

#Check Tweets that are not influenced by other tweets
tweetsEn.groupBy("influencer").count().sort(col("count").desc()).show(10)

#Check tweets that quotes other people
tweetsEn.groupBy("is_quote_status").count().sort(col("count").desc()).show()

#Check if influence data makes sense
tweetsEn.groupBy("reply_count").count().sort(col("count").desc()).show(10)
tweetsEn.groupBy("retweet_count").count().sort(col("count").desc()).show(10)
tweetsEn.groupBy("favorite_count").count().sort(col("count").desc()).show(10)

+-------------------+-----+
|     original_tweet|count|
+-------------------+-----+
|               null|20596|
|1205493508511059968|    3|
|1205501506692304896|    2|
|1205510619128041474|    2|
|1205494373905031172|    2|
|1205516927550066690|    2|
|1205500301421056002|    2|
|1202777589674336256|    1|
|1205496352588881920|    1|
|1205495271448285186|    1|
+-------------------+-----+
only showing top 10 rows

+------------------+-----+
|        influencer|count|
+------------------+-----+
|              null|20443|
|        3131144855|   20|
|         164226176|    7|
|         117777690|    5|
|         856010760|    5|
|          80802900|    4|
|          14291684|    4|
|          65045121|    4|
|745370688375099392|    3|
|           5402612|    3|
+------------------+-----+
only showing top 10 rows

+---------------+-----+
|is_quote_status|count|
+---------------+-----+
|           true|20811|
+---------------+-----+

+-------------------+-----+
|        reply_count|count|
+

In [6]:
#drop columns
tweetsDC = tweetsEn.drop('retweet_count', 'favorite_count', 'coordinates', 'tweet_date', 'is_quote_status')
#tweetsDC.printSchema()
#drop users with less than 50 followers
tweetsFC = tweetsDC.where("followers_count>=50")
tweetsFC.groupBy("tweet_lang").count().sort(col("count").desc()).show(10)
tweetsFC.groupBy("followers_count").count().sort(col("count").desc()).show(10)
tweetsFC.groupBy("is_verified").count().show()

+----------+-----+
|tweet_lang|count|
+----------+-----+
|        en|19116|
+----------+-----+

+---------------+-----+
|followers_count|count|
+---------------+-----+
|            186|   91|
|             52|   43|
|             54|   38|
|            155|   36|
|             75|   36|
|             65|   35|
|             66|   35|
|             73|   34|
|            147|   33|
|            154|   32|
+---------------+-----+
only showing top 10 rows

+-----------+-----+
|is_verified|count|
+-----------+-----+
|       true|  297|
|      false|18819|
+-----------+-----+



In [7]:
tweetsFC.groupBy("date").count().sort(col("count").desc()).show(20)
#Choose tweets of users who were created before Dec 2019 
tweetsNC = tweetsFC.where("date NOT LIKE '%2019' OR date NOT LIKE '%Dec%'")

#Choose tweets of users who were created before Nov 2019 
tweetsNC = tweetsNC.where("date NOT LIKE '%2019' OR date NOT LIKE '%Nov%'")
#tweetsNC.groupBy("tweet_lang").count().sort(col("count").desc()).show()
tweetsNC.groupBy("reply_count").count().sort(col("count").desc()).show()
tweetsNC = tweetsNC.drop('tweet_lang')
tweetsNC.printSchema()


+--------------------+-----+
|                date|count|
+--------------------+-----+
|Fri May 24 17:22:...|   76|
|Fri Jul 13 22:29:...|   24|
|Thu Aug 08 12:56:...|   23|
|Wed Jul 08 14:08:...|   16|
|Sat Jan 26 08:59:...|   16|
|Mon Jul 09 11:00:...|   15|
|Thu Jun 02 14:10:...|   15|
|Fri Apr 20 10:49:...|   15|
|Sun Feb 06 10:37:...|   15|
|Fri Aug 05 13:14:...|   15|
|Mon Jan 15 12:51:...|   14|
|Sat Nov 03 10:44:...|   14|
|Wed Mar 25 02:53:...|   13|
|Thu Dec 11 22:59:...|   13|
|Fri Aug 03 08:13:...|   13|
|Tue Feb 28 19:33:...|   13|
|Thu Jan 12 17:43:...|   11|
|Mon Sep 09 15:57:...|   11|
|Tue Feb 23 01:40:...|   11|
|Wed Aug 29 20:27:...|   10|
+--------------------+-----+
only showing top 20 rows

+-------------------+-----+
|        reply_count|count|
+-------------------+-----+
|1205368801438707713| 1952|
|1205441159805444098|  969|
|1205326557474103296|  695|
|1203927100740243456|  427|
|1205403440144429056|  343|
|1205421989189566465|  253|
|1205408483287457793|  206

# Basic Analysis

In [8]:
#What is trending right now? top 20 popular hashtags 

tweetsNC.groupBy("is_verified").count().show()

tweetsNC.select(explode(col("hashtags")).alias("hashtags_list"))\
            .groupBy("hashtags_list.text")\
            .count().sort(col("count").desc())\
            .show(20)

    
#How many users?    
tweetsNC.groupBy("user_name").count().sort(col("count").desc()).show(10)

#tweetsNC.describe(['userId']).show()
#print("---------------------number of tweets --------------------------------")
tweetsNC.agg(count("userId")).show()
#print("---------------------User of distinct users --------------------------------")
tweetsNC.agg(countDistinct("userId")).show()


#top 20 popular Tweets
    
#tweets with emotions 
#tweetsNC.filter(col("text").like(":")).show(10)
#tweetsNC.printSchema()

+-----------+-----+
|is_verified|count|
+-----------+-----+
|       true|  297|
|      false|18622|
+-----------+-----+

+-------------------+-----+
|               text|count|
+-------------------+-----+
|             Brexit|  474|
|                 UK|  104|
|              leave|   64|
|            fishing|   63|
|         reclaiming|   62|
|             GE2019|   62|
|    GeneralElection|   55|
|             brexit|   51|
|          Trump2020|   39|
|    BritishElection|   38|
|            goodbye|   37|
|          socialist|   37|
|      GetBrexitDone|   37|
|                NWO|   37|
|          GoodGrief|   37|
|       BorisJohnson|   37|
|            liberal|   37|
|GeneralElection2019|   36|
|           IndyRef2|   34|
|     UKelection2019|   30|
+-------------------+-----+
only showing top 20 rows

+--------------+-----+
|     user_name|count|
+--------------+-----+
|   TaraJewell6|   76|
|cloudwanderer3|   24|
|   SaraPadmore|   23|
|Carter7Raymond|   16|
|    z_chrissie|   1

In [13]:
tweetsNC = tweetsNC.drop('place', 'timestamps', 'user_name', 'influencer', 'timestamp', 'user_descr', 'original_tweet', 'date', 'quoted_status_id')
#Replace Missing Values and then remove all tweets from users in locations outside the UK
tweetClean = tweetsNC.na.fill({"location":"ND"})

tweetCleans = tweetClean.where("location NOT LIKE '%USA%'")


tweetCleans = tweetCleans.withColumn('location', when(tweetCleans.location.like('%England%'), 'England').otherwise(tweetCleans['location']))
tweetCleans = tweetCleans.withColumn('location', when(tweetCleans.location.like('%Wales%'), 'Wales').otherwise(tweetCleans['location']))
tweetCleans = tweetCleans.withColumn('location', when(tweetCleans.location.like('%Scotland%'), 'Scotland').otherwise(tweetCleans['location']))
tweetCleans = tweetCleans.withColumn('location', when(tweetCleans.location.like('%Northern Ireland%'), 'NI').otherwise(tweetCleans['location']))

tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'United Kingdom', 'UK'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Great Britain', 'UK'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'uk', 'UK'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'London, UK', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Manchester', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'London ', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'London', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Yorkshire', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Liverpool', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'london', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Cornwall', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Sheffield', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Bristol', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Essex', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'oxford', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Sussex', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Derby', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'West Sussex', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Leeds', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Oxfordshire', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Surrey', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Oxford', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Kent', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Birmingham', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Brighton', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Cambridge, UK', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Hampshire, UK', 'England'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Glasgow', 'Scotland'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Edinburgh', 'Scotland'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Cardiff', 'Wales'))
tweetCleans = tweetCleans.withColumn('location', regexp_replace('location', 'Belfast', 'NI'))
#tweetsClean = tweetCleans.select("*").where("location LIKE 'UK' OR location LIKE 'UK' OR location LIKE 'England' OR location LIKE 'NI' OR location LIKE 'Scotland' OR location LIKE 'Wales' ")

In [15]:
tweetsClean = tweetCleans
tweetsClean.agg(count("location")).show()
tweetsClean.agg(countDistinct("location")).show()
tweetsClean.groupBy("location").count().sort(col("count").desc()).show(100)
tweetsClean.show(2, False)

+---------------+
|count(location)|
+---------------+
|          18240|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    4683|
+------------------------+

+--------------------+-----+
|            location|count|
+--------------------+-----+
|                  ND| 5382|
|             England| 3409|
|                  UK|  688|
|            Scotland|  478|
|               Wales|  237|
|       United States|  194|
|         England, UK|   76|
|                  NI|   58|
|      Washington, DC|   46|
|        New York, NY|   36|
|             Ireland|   31|
|               Earth|   31|
|              Canada|   28|
|        West England|   24|
|     Laurelindórenan|   24|
|        Planet Earth|   23|
|    Toronto, Ontario|   22|
|          England UK|   22|
|               Texas|   22|
|            England |   22|
|              Europe|   22|
|         Chicago, IL|   22|
|      European Union|   21|
|            New Yo

# Filtering tweets

In [16]:
tweetsClean.coalesce(1).write.json("file:///Users/Laith/Downloads/features_tweets.json")

In [2]:
#Load firstClean tweets 
firstClean = spark.read.json("file:///Users/Laith/Downloads/firstClean.json")

#Print the schema/data structure in a tree format
firstClean.printSchema()

root
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- is_verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- userId: long (nullable = true)



In [3]:
firstClean = firstClean.where("location NOT LIKE '%Dublin%'")
firstClean = firstClean.where("location NOT LIKE '%ND%'")
firstClean = firstClean.where("location NOT LIKE '%France%'")
firstClean = firstClean.where("location NOT LIKE '%United States%'")
firstClean = firstClean.where("location NOT LIKE 'Ireland%'")
firstClean = firstClean.where("location NOT LIKE '%Washington%'")
firstClean = firstClean.where("location NOT LIKE '%New York%'")
firstClean = firstClean.where("location NOT LIKE '%Canada%'")
firstClean = firstClean.where("location NOT LIKE '%Chicago%'")
firstClean = firstClean.where("location NOT LIKE '%Texas%'")
firstClean = firstClean.where("location NOT LIKE '%Europe%'")
firstClean = firstClean.where("location NOT LIKE '%Toronto%'")
firstClean = firstClean.where("location NOT LIKE '%Earth%'")
firstClean = firstClean.where("location NOT LIKE '%Laurelindórenan%'")
firstClean = firstClean.where("location NOT LIKE '%India%'")
firstClean = firstClean.where("location NOT LIKE '%Houston%'")
firstClean = firstClean.where("location NOT LIKE '%Switzerland%'")
firstClean = firstClean.where("location NOT LIKE '%Germany%'")
firstClean = firstClean.where("location NOT LIKE '%Italy%'")
firstClean = firstClean.where("location NOT LIKE '%Los Angeles%'")
firstClean = firstClean.where("location NOT LIKE '%Belgium%'")
firstClean = firstClean.where("location NOT LIKE '%dreams%'")
firstClean = firstClean.where("location NOT LIKE '%Neverland%'")
firstClean = firstClean.where("location NOT LIKE '%NYC%'")
firstClean = firstClean.where("location NOT LIKE '%Berlin%'")
firstClean = firstClean.where("location NOT LIKE '%Tarragona%'")
firstClean = firstClean.where("location NOT LIKE '%TX%'")
firstClean = firstClean.where("location NOT LIKE '%NY%'")
firstClean = firstClean.where("location NOT LIKE '%CA%'")
firstClean = firstClean.where("location NOT LIKE '%Deutschland%'")
firstClean = firstClean.where("location NOT LIKE '%Hong Kong%'")
firstClean = firstClean.where("location NOT LIKE '%she/her%'")
firstClean = firstClean.where("location NOT LIKE '%AZ%'")
firstClean = firstClean.where("location NOT LIKE '%At the%'")
firstClean = firstClean.where("location NOT LIKE '%EU%'")
firstClean = firstClean.where("location NOT LIKE '%PA%'")
firstClean = firstClean.where("location NOT LIKE '%Everywhere%'")
firstClean = firstClean.where("location NOT LIKE '%Finland%'")
firstClean = firstClean.where("location NOT LIKE '%Boston%'")
firstClean = firstClean.where("location NOT LIKE '%FL%'")
firstClean = firstClean.where("location NOT LIKE '%California%'")
firstClean = firstClean.where("location NOT LIKE '%WA%'")
firstClean = firstClean.where("location NOT LIKE '%Always%'")
firstClean = firstClean.where("location NOT LIKE '%Brussels%'")
firstClean = firstClean.where("location NOT LIKE '%Coming for%'")
firstClean = firstClean.where("location NOT LIKE '%Australia%'")
firstClean = firstClean.where("location NOT LIKE '%Mars%'")
firstClean = firstClean.where("location NOT LIKE '%MD%'")
firstClean = firstClean.where("location NOT LIKE '%Global%'")
firstClean = firstClean.where("location NOT LIKE '%Forever%'")
firstClean = firstClean.where("location NOT LIKE '%NV%'")
firstClean = firstClean.where("location NOT LIKE '%Illinois%'")
firstClean = firstClean.where("location NOT LIKE '%KY%'")
firstClean = firstClean.where("location NOT LIKE '%NoNazi%'")


In [4]:

firstClean.agg(count("location")).show()
firstClean.agg(countDistinct("location")).show()
firstClean.groupBy("location").count().sort(col("count").desc()).show(20)

+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3923|
+------------------------+

+--------------------+-----+
|            location|count|
+--------------------+-----+
|             England| 3409|
|                  UK|  688|
|            Scotland|  478|
|               Wales|  237|
|         England, UK|   76|
|                  NI|   58|
|        West England|   24|
|          England UK|   22|
|            England |   22|
|       South England|   19|
|           EnglandUK|   16|
|    Hackney, England|   14|
|        East England|   13|
|     England/England|   13|
|             Cumbria|    9|
|    Croydon, England|    9|
|Long Compton, Sou...|    9|
|  Islington, England|    9|
|Kingston upon Tha...|    9|
|Caterham, England...|    9|
+--------------------+-----+
only showing top 20 rows



In [7]:
#firstClean = firstClean.withColumn('location', when(firstClean.location.like('%England%'), 'England').otherwise(firstClean['location']))
#firstClean = firstClean.withColumn('location', when(firstClean.location.like('%Wales%'), 'Wales').otherwise(firstClean['location']))
#firstClean = firstClean.withColumn('location', when(firstClean.location.like('%Scotland%'), 'Scotland').otherwise(firstClean['location']))
#firstClean = firstClean.withColumn('location', when(firstClean.location.like('%Northern Ireland%'), 'NI').otherwise(firstClean['location']))
#firstClean = firstClean.withColumn('location', when(firstClean.location.like('%NI%'), 'NI').otherwise(firstClean['location']))
#firstClean = firstClean.withColumn('location', when(firstClean.location.like('%UK%'), 'UK').otherwise(firstClean['location']))
#firstClean = firstClean.withColumn('location', when(firstClean.location.like('%uk%'), 'UK').otherwise(firstClean['location']))

firstClean.agg(count("location")).show()
firstClean.agg(countDistinct("location")).show()
firstClean.groupBy("location").count().sort(col("count").desc()).show(100, False)

+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3193|
+------------------------+

+--------------------------+-----+
|location                  |count|
+--------------------------+-----+
|England                   |4365 |
|UK                        |1034 |
|Scotland                  |501  |
|Wales                     |245  |
|NI                        |77   |
|Cumbria                   |9    |
|Long Compton, South Warks |9    |
|York                      |9    |
|Nottingham                |8    |
|Devon                     |8    |
|Hull                      |8    |
|manchester                |8    |
|Wakefield                 |8    |
|Shropshire                |8    |
|Newcastle upon Tyne       |8    |
|Hastings                  |8    |
|People's Republic of China|8    |
|Tx                        |7    |
|Whitstable                |7    |
|So

In [8]:
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Nottingham', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'manchester', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Wakefield', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Hull', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'York', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Shropshire', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Long Compton, South Warks', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Cumbria', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Devon', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Newcastle upon Tyne', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Hastings', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'hull', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'wigan', 'England'))
firstClean = firstClean.withColumn('location', when(firstClean.location.like('%england%'), 'England').otherwise(firstClean['location']))

In [9]:
firstClean.agg(count("location")).show()
firstClean.agg(countDistinct("location")).show()
firstClean.groupBy("location").count().sort(col("count").desc()).show(100, False)

+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3167|
+------------------------+

+------------------------------+-----+
|location                      |count|
+------------------------------+-----+
|England                       |4489 |
|UK                            |1034 |
|Scotland                      |501  |
|Wales                         |245  |
|NI                            |77   |
|People's Republic of China    |8    |
|Dorset                        |7    |
|Whitstable                    |7    |
|America                       |7    |
|Michigan                      |7    |
|The Netherlands               |7    |
| Dorset, Ireland, Spain       |7    |
|Paris                         |7    |
|Up North                      |7    |
|Florida                       |7    |
|South Carolina                |7    |
|Melbourne, Victoria           |

In [12]:
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Whitestable', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Dorest', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Cheshire', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Isle of Wight', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Southampton', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Suffolk', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Basingstoke, South Warks', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Bournemouth', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Epsom', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Worcestershire', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'English', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Aberdeenshire', 'Scotland'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'glasgow', 'Scotland'))
firstClean = firstClean.withColumn('location', when(firstClean.location.like('%England%'), 'England').otherwise(firstClean['location']))
firstClean = firstClean.withColumn('location', when(firstClean.location.like('%Uk%'), 'UK').otherwise(firstClean['location']))

In [14]:
firstClean.show(10, False)
#firstClean.agg(count("location")).show()
#firstClean.agg(countDistinct("location")).show()
#firstClean.groupBy("location").count().sort(col("count").desc()).show(100, False)

+---------------+-------------+--------+-----------+----------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|followers_count|friends_count|hashtags|is_verified|location  |reply_count        |text                                                                                                                                        |userId             |
+---------------+-------------+--------+-----------+----------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|2387           |4996         |[]      |false      |England   |1205470817959776257|Urgh                                                                                                                                        |351883595          |
|326            |403

In [15]:
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Cheshire', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Isle of Wight', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Southampton', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Suffolk', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Basingstoke, South Warks', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Bournemouth', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Epsom', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Worcestershire', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'English', 'England'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'Aberdeenshire', 'Scotland'))
firstClean = firstClean.withColumn('location', regexp_replace('location', 'glasgow', 'Scotland'))
firstClean = firstClean.withColumn('location', when(firstClean.location.like('%England%'), 'England').otherwise(firstClean['location']))
firstClean = firstClean.withColumn('location', when(firstClean.location.like('%Uk%'), 'UK').otherwise(firstClean['location']))
firstClean.coalesce(1).write.json("file:///Users/Laith/Downloads/features_tweets2.json")

In [29]:
#Load secondClean tweets 
secondClean = spark.read.json("file:///Users/Laith/Downloads/secondClean.json")

#Print the schema/data structure in a tree format
secondClean.printSchema()



root
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- is_verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- userId: long (nullable = true)



In [30]:
secondClean.agg(count("location")).show()
secondClean.agg(countDistinct("location")).show()
secondClean.groupBy("location").count().sort(col("count").desc()).show(100, False)

+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3095|
+------------------------+

+------------------------------+-----+
|location                      |count|
+------------------------------+-----+
|England                       |4617 |
|UK                            |1047 |
|Scotland                      |511  |
|Wales                         |245  |
|NI                            |77   |
|People's Republic of China    |8    |
|Florida                       |7    |
|Whitstable                    |7    |
|Michigan                      |7    |
| Dorset, Ireland, Spain       |7    |
|Up North                      |7    |
|America                       |7    |
|The Netherlands               |7    |
|Paris                         |7    |
|Melbourne, Victoria           |7    |
|South Carolina                |7    |
|Dorset                        |

In [31]:
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Whitstable', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Dorset', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Inverness', 'Scotland'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Berkshire', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Basingstoke', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Wirral', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Croydon', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Northumberland', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Coningsby', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Finsbury', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Herefordshire', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Taunton', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'southampton', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Swindon', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Pembrokeshire', 'England'))
secondClean = secondClean.withColumn('location', regexp_replace('location', 'Gibraltar', 'UK'))


In [32]:
secondClean.agg(count("location")).show()
secondClean.agg(countDistinct("location")).show()
secondClean.groupBy("location").count().sort(col("count").desc()).show(100, False)

+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3081|
+------------------------+

+-----------------------------+-----+
|location                     |count|
+-----------------------------+-----+
|England                      |4675 |
|UK                           |1051 |
|Scotland                     |517  |
|Wales                        |245  |
|NI                           |77   |
|People's Republic of China   |8    |
|Florida                      |7    |
|Michigan                     |7    |
|Up North                     |7    |
|Paris                        |7    |
|America                      |7    |
|The Netherlands              |7    |
|South Carolina               |7    |
|Tx                           |7    |
|Lagos, Nigeria               |7    |
| England, Ireland, Spain     |7    |
|Melbourne, Victoria          |7    |
|Here        

In [33]:
secondClean = secondClean.withColumn('location', when(secondClean.location.like('%England%'), 'England').otherwise(secondClean['location']))
secondClean = secondClean.withColumn('location', when(secondClean.location.like('%united kingdom%'), 'UK').otherwise(secondClean['location']))
secondClean = secondClean.withColumn('location', when(secondClean.location.like('%wales%'), 'Wales').otherwise(secondClean['location']))

secondClean.agg(count("location")).show()
secondClean.agg(countDistinct("location")).show()
secondClean.groupBy("location").count().sort(col("count").desc()).show(100, False)




+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3056|
+------------------------+

+------------------------------+-----+
|location                      |count|
+------------------------------+-----+
|England                       |4721 |
|UK                            |1056 |
|Scotland                      |517  |
|Wales                         |252  |
|NI                            |77   |
|People's Republic of China    |8    |
|Florida                       |7    |
|Up North                      |7    |
|America                       |7    |
|South Carolina                |7    |
|Melbourne, Victoria           |7    |
|The Netherlands               |7    |
|Paris                         |7    |
|Lagos, Nigeria                |7    |
|Tx                            |7    |
|Michigan                      |7    |
|Russia                        |

In [35]:
secondClean.coalesce(1).write.json("file:///Users/Laith/Downloads/features_tweets3.json")

In [42]:
#Load secondClean tweets 
thirdClean = spark.read.json("file:///Users/Laith/Downloads/thirdClean.json")

#Print the schema/data structure in a tree format
thirdClean.printSchema()

thirdClean.agg(count("location")).show()
thirdClean.agg(countDistinct("location")).show()
thirdClean.groupBy("location").count().sort(col("count").desc()).show(100, False)

thirdClean = thirdClean.withColumn('location', when(thirdClean.location.like('%Britain%'), 'UK').otherwise(thirdClean['location']))
thirdClean = thirdClean.withColumn('location', when(thirdClean.location.like('%britain%'), 'UK').otherwise(thirdClean['location']))
thirdClean = thirdClean.withColumn('location', when(thirdClean.location.like('%BRITAIN%'), 'UK').otherwise(thirdClean['location']))

root
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- is_verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- userId: long (nullable = true)

+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3056|
+------------------------+

+------------------------------+-----+
|location                      |count|
+------------------------------+-----+
|England                       |4721 |
|UK                            |1056 |
|Scotland                      |517  |
|Wales     

In [43]:
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Staffordshire', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Reading', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Portsmouth', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'chesterfield', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Wokingham', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'isle of wight', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Elephant & Castle', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Cambridgeshire', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Brentwood, East', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Sunderland', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'newcastle upon tyne', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Exeter', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Hampshire mostly', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Gloucestershire', 'England'))
thirdClean = thirdClean.withColumn('location', regexp_replace('location', 'Norfolk', 'England'))

thirdClean = thirdClean.withColumn('location', when(thirdClean.location.like('%England%'), 'England').otherwise(thirdClean['location']))
thirdClean = thirdClean.withColumn('location', when(thirdClean.location.like('%scotland%'), 'Scotland').otherwise(thirdClean['location']))
thirdClean = thirdClean.withColumn('location', when(thirdClean.location.like('%Scotland%'), 'Scotland').otherwise(thirdClean['location']))


In [44]:
thirdClean.agg(count("location")).show()
thirdClean.agg(countDistinct("location")).show()
thirdClean.groupBy("location").count().sort(col("count").desc()).show(200, False)

+---------------+
|count(location)|
+---------------+
|          10780|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                    3004|
+------------------------+

+-----------------------------------------------------+-----+
|location                                             |count|
+-----------------------------------------------------+-----+
|England                                              |4808 |
|UK                                                   |1076 |
|Scotland                                             |524  |
|Wales                                                |252  |
|NI                                                   |77   |
|People's Republic of China                           |8    |
|South Carolina                                       |7    |
|Up North                                             |7    |
|America                                              |7    |
|Florida                     

In [45]:
thirdClean.coalesce(1).write.json("file:///Users/Laith/Downloads/features_tweets4.json")

In [2]:
#Load secondClean tweets 
cleanData = spark.read.json("file:///Users/Laith/Downloads/fourthClean.json")

#Print the schema/data structure in a tree format
#cleanData.printSchema()

In [3]:
cleanData = cleanData.select("*").where("location LIKE 'UK' OR location LIKE 'England' OR location LIKE 'NI' OR location LIKE 'Scotland' OR location LIKE 'Wales'")
cleanData.agg(count("location")).show()
cleanData.agg(countDistinct("location")).show()
cleanData.groupBy("location").count().sort(col("count").desc()).show()

+---------------+
|count(location)|
+---------------+
|           6737|
+---------------+

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                       5|
+------------------------+

+--------+-----+
|location|count|
+--------+-----+
| England| 4808|
|      UK| 1076|
|Scotland|  524|
|   Wales|  252|
|      NI|   77|
+--------+-----+



In [4]:
cleanData = cleanData.withColumnRenamed('reply_count', 'reply_to')
cleanData.printSchema()

root
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- is_verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_to: long (nullable = true)
 |-- text: string (nullable = true)
 |-- userId: long (nullable = true)



In [6]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StopWordsRemover
print("---------------------Tokenizer--------------------------------")
#tokenizer = Tokenizer(inputCol="text", outputCol="words")
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\\W")
countTokens = udf(lambda words: len(words), IntegerType())
#tokenized = tokenizer.transform(tweetsDF)
#tokenized.show(2, False)
regexTokenized = regexTokenizer.transform(cleanData)
regexTokenized.show(2, False)

##I NEED ENOUGH MEMORY FOR THIS
#regexTokenized.select("text", "words").withColumn("tokens", countTokens(col("words"))).show(10, False)

---------------------Tokenizer--------------------------------
+---------------+-------------+--------+-----------+--------+-------------------+---------------------------------------------------------------+------------------+---------------------------------------------------------------------+
|followers_count|friends_count|hashtags|is_verified|location|reply_to           |text                                                           |userId            |words                                                                |
+---------------+-------------+--------+-----------+--------+-------------------+---------------------------------------------------------------+------------------+---------------------------------------------------------------------+
|2387           |4996         |[]      |false      |England |1205470817959776257|Urgh                                                           |351883595         |[urgh]                                                              

In [8]:
from pyspark.ml.feature import StopWordsRemover
print("---------------------Stop Word Remover --------------------------------")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
#remover.transform(tokenized).show(2, False)   
cleanTweetsDF=remover.transform(regexTokenized) 



---------------------Stop Word Remover --------------------------------


In [9]:
cleanTweetsDF.printSchema()

root
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- is_verified: boolean (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_to: long (nullable = true)
 |-- text: string (nullable = true)
 |-- userId: long (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
cleanData = cleanTweetsDF.drop('hashtags', 'is_verified', 'words', 'text')
cleanData.printSchema()
cleanData.count()

root
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_to: long (nullable = true)
 |-- userId: long (nullable = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)



6737

In [14]:
cleanData.coalesce(1).write.json("file:///Users/Laith/Downloads/features_tweets5.json")